In [1]:
# Read data
import csv
import cv2
import numpy as np

# Helper for loading image from given path and dataset
def read_img(source_path, dataset):
    filename = source_path.split('/')[-1]
    current_path = './' + dataset + '/IMG/' + filename
    image = cv2.imread(current_path)
    return image

# List of datasets that are loaded, list of directories
datasets = ['normal', 'back-to-road', 'bridge']

images = []
steering = []

# Load datasets
for dataset in datasets:
    with open('./' + dataset + '/driving_log.csv') as csvfile:
        reader = csv.reader(csvfile)
        for line in reader:
            center_image = read_img(line[0], dataset)
            left_image = read_img(line[1], dataset)
            right_image = read_img(line[2], dataset)

            images.append(center_image)
            images.append(left_image)
            images.append(right_image)

            steering_correction = 0.2
            steering_center = float(line[3])
            steering_left = steering_center + steering_correction
            steering_right = steering_center - steering_correction

            steering.append(steering_center)
            steering.append(steering_left)
            steering.append(steering_right)
        print('finished loading dataset: ', dataset)
    
X_train = np.array(images)
y_train = np.array(steering)

print("X_train shape: ", X_train.shape)
print("y_train shape: ", y_train.shape)
    

finished dataset:  track2
X_train shape:  (11499, 160, 320, 3)
y_train shape:  (11499,)


In [5]:
# Network Architecture
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Dropout
from keras.layers import Convolution2D, Cropping2D

model = Sequential()

## Preprocessing
model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((70,25),(0,0))))

## Convolutional layers
model.add(Convolution2D(24,5,5,subsample=(2,2), activation="relu"))
model.add(Convolution2D(36,5,5,subsample=(2,2),activation="relu"))
model.add(Convolution2D(48,5,5,subsample=(2,2),activation="relu"))
model.add(Convolution2D(64,3,3,activation="relu"))
model.add(Convolution2D(64,3,3,activation="relu"))

## Fully connected layers
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(100))
model.add(Dropout(0.5))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))

## Train
model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, epochs=5)

model.save('model.h5')
print('model saved')

/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(24, (5, 5), activation="relu", strides=(2, 2))`
  # This is added back by InteractiveShellApp.init_path()
/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(36, (5, 5), activation="relu", strides=(2, 2))`
  if sys.path[0] == '':
/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (5, 5), activation="relu", strides=(2, 2))`
  del sys.path[0]
/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  
/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_laun

Train on 9199 samples, validate on 2300 samples
Epoch 1/5
9199/9199 [==============================] - 17s 2ms/step - loss: 0.1267 - val_loss: 0.1268
Epoch 2/5
9199/9199 [==============================] - 17s 2ms/step - loss: 0.1006 - val_loss: 0.0984
Epoch 3/5
9199/9199 [==============================] - 17s 2ms/step - loss: 0.0870 - val_loss: 0.0964
Epoch 4/5
9199/9199 [==============================] - 17s 2ms/step - loss: 0.0777 - val_loss: 0.1070
Epoch 5/5
9199/9199 [==============================] - 17s 2ms/step - loss: 0.0720 - val_loss: 0.1107
model saved
